hello
os library only imported to find the current working folder to relocate the DB file.
SQLalchemy is imported to make SQL queries from the database containing the simulation results.
We are also importing Pandas to read the result of SQL queries into Dataframes for further analysis.

In [1]:
import os 
import pandas as pd
from sqlalchemy import create_engine

The results database that we are going to read and analyze data from is created by the other project in my account: [Feeder Power Loss Simulation](https://github.com/pooya-bagheri/Feeder-PowerLoss-Simulation)

In [2]:
DBfilePath=os.path.abspath('')+'//'+'SimResultsIEEE123NodesCase2.db'
DB = create_engine('sqlite:///'+DBfilePath, echo=False)   

Database includes data for 30 simulated days. We don't want to explode the memory.So, Let's just focus on the first day read first day's Ploss data and voltages for one node:

In [3]:
Ploss=pd.read_sql_query('Select * from Losses',con=DB)
Ploss.head()

,InstantID,Ploss
0,1,25.598380
1,2,25.351119
2,3,25.104456
3,4,24.859135
4,5,24.615373


In [4]:
Ploss=pd.read_sql_query('''
select Instants.InstantID,Losses.Ploss
from Instants join Losses on Instants.InstantID=Losses.InstantID
where Instants.Day==1''',con=DB)
Ploss.head()
Ploss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
InstantID    1440 non-null int64
Ploss        1440 non-null float64
dtypes: float64(1), int64(1)
memory usage: 22.6 KB


In [5]:
Voltages=pd.read_sql_query('''
select Instants.InstantID,Voltages.NodeID,Voltages.Vmag
from Instants join Voltages on Instants.InstantID=Voltages.InstantID
where Instants.Day==1''',con=DB)
Voltages.head()

,InstantID,NodeID,Vmag
0,1,1,1.0000
1,1,2,1.0125
2,1,3,1.0125
3,1,4,1.0069
4,1,5,1.0132


We can see that voltages of all node are present here. Let's group voltage measurements to get the average by every node.

In [7]:
VoltagesMean=pd.read_sql_query('''
select Instants.InstantID,avg(Voltages.Vmag)
from Instants join Voltages on Instants.InstantID=Voltages.InstantID
where Instants.Day==1
group by Instants.InstantID''',con=DB)
VoltagesMean.head()

,InstantID,avg(Voltages.Vmag)
0,1,1.013455
1,2,1.013608
2,3,1.013768
3,4,1.013920
4,5,1.014074


Let's have a look at what we have collected so far: